### kaggle資料集 : Brain MRI Images for Brain Tumor Detection
### 資料集解釋 : 總共253個檔案，其中no有98個檔案，Yes有155個檔案
### [資料鏈接][1]
### [參考寫法][2]

[1]:<https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection?select=yes> "Brain MRI資料鏈接"
[2]: <https://www.kaggle.com/code/aissaasasd/brain-tumor-detector-92> "參考寫法"

In [2]:
import cv2
from glob import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

### 資料預處理

In [3]:
(img_height, img_width) = 224,224
data_dir = "./brain_tumor_dataset"
folders = glob(data_dir+"/*")
print(len(folders),folders)

3 ['./brain_tumor_dataset/valid', './brain_tumor_dataset/test', './brain_tumor_dataset/train']


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=15,
                                   zoom_range=0.4,
                                   horizontal_flip=True,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_set = train_datagen.flow_from_directory(data_dir+"/train",
                                                 target_size=(img_height, img_width),batch_size=32,class_mode="categorical")

Found 166 images belonging to 2 classes.


In [6]:
test_set = test_datagen.flow_from_directory(data_dir+"/test",
                                            target_size=(img_height,img_width),
                                            batch_size=32,
                                            class_mode='categorical')

Found 56 images belonging to 2 classes.


In [9]:
# 設定模型
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',padding='same',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation="relu",padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# model.add(Conv2D(128,(3,3),activation="relu",padding='same'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(256,(3,3),activation="relu",padding='same'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2,activation="softmax"))
model.summary()
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# model-checkpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")

history = model.fit(train_set,validation_data=test_set,epochs=30,batch_size=64,verbose=2, callbacks=[early_stopping])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 224, 224, 128)     3584      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 128)    0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 112, 112, 128)     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      73792     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, 56, 56, 64)       

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()